In [5]:
#Projeto elaborado para a coleta de infos disponíveis numa API pública, que coleta dados de CNPJs, como Razão Social, nome dos sócios e Regime Tributário.#Aqui, a ideia é coletar as informações supracitadas e realizar o envio por emailimport pandas as pd
import requests
from time import sleep
base = pd.read_clipboard()
from tqdm import tqdm

cnpjs = []
supplier = []
simples = []
pessoa_fisica = []
tipo = []

for i in tqdm(range(0, len(base))):
    cnpj = (base['cnpj'][i])
    try:
        cnpjs.append(cnpj)
        r = requests.get('https://publica.cnpj.ws/cnpj/{}'.format(cnpj))
        a = r.json()
        try:
            simples.append(a['simples']['simples'])
            try:
                supplier.append(a['razao_social'])
            except:
                supplier.append('Não informado')
            try:
                pessoa_fisica.append(a['socios'][0]['nome'])
            except:
                pessoa_fisica.append('Não informado')
            try:
                tipo.append(a['natureza_juridica']['descricao'])
            except:
                tipo.append('Não informado')
        except:
            simples.append('Não')
            try:
                supplier.append(a['razao_social'])
            except:
                supplier.append('Não informado')
            
            try:
                pessoa_fisica.append(a['socios'][0]['nome'])
            except:
                pessoa_fisica.append('Não informado')
            
            try:
                tipo.append(a['natureza_juridica']['descricao'])
            except:
                tipo.append('Não informado')
        sleep(20)        
    except:
        simples.append('CNPJ Não encontrado')
        supplier.append('CNPJ Não encontrado')
        pessoa_fisica.append('CNPJ Não encontrado')
        tipo.append('CNPJ Não encontrado')          
    
    
export = pd.DataFrame(columns = ['CNPJ', 'Supplier', 'Tipo', 'Pessoa Física', 'Simples Nacional'])

export['CNPJ'] = cnpjs
export['Supplier'] = supplier
export['Tipo'] = tipo
export['Pessoa Física'] = pessoa_fisica
export['Simples Nacional'] = simples

export.to_excel('Export API.xlsx')

100%|███████████████████████████████████████████████████████████████████████████| 2439/2439 [14:31:19<00:00, 21.43s/it]


In [2]:
# Importar bibliotecas
import smtplib, ssl
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication #Biblioteca para inserir anexos

# Definindo o documento HTML
html = '''
    <html>
        <body>
            <h1>Email teste</h1>
            <p>Inserir aqui o conteúdo textual</p>
        </body>
    </html>
    '''

# Criando função para anexar o(s) arquivo(s) ao email
def attach_file_to_email(email_message, filename):

    with open(filename, "rb") as f:
        file_attachment = MIMEApplication(f.read())

    file_attachment.add_header(
        "Content-Disposition",
        f"attachment; filename= {filename}",
    )
    # Anexando arquivo a mensagem
    email_message.attach(file_attachment)



# Configurando email e senha de envio e email de destino
email_from = 'envio@seuemail.com.br'
password = 'xxxxxxxxxxxxxxxx'
email_to = 'recebedor@seuemail.com.br'

# Coletando a data atual para inserir ao email
date_str = pd.Timestamp.today().strftime('%Y-%m-%d')

# Criando um MIMEMultipart class, e configurando seu remetente, destinatário e assunto
email_message = MIMEMultipart()
email_message['From'] = email_from
email_message['To'] = email_to
email_message['Subject'] = f'Report email - {date_str}'

# Inserindo a mensagem html criada anteriormente no tipo Texto MIME
email_message.attach(MIMEText(html, "html"))

attach_file_to_email(email_message, 'Export API.xlsx')

# Convertendo em string
email_string = email_message.as_string()

# Conectando com o servidor Gmail SMTP e enviando
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(email_from, password)
    server.sendmail(email_from, email_to, email_string)
    
print('Fim')

Fim
